In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import resnet_model
import os

%matplotlib inline

/home/barney_awesome_li/anaconda3/envs/tfcpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
# Load training data set
train = pd.read_json('./data/train.json')

In [4]:
print(train.head())

                                              band_1  \
0  [-27.878360999999998, -27.15416, -28.668615, -...   
1  [-12.242375, -14.920304999999999, -14.920363, ...   
2  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2        id inc_angle  \
0  [-27.154118, -29.537888, -31.0306, -32.190483,...  dfd5f913   43.9239   
1  [-31.506321, -27.984554, -26.645678, -23.76760...  e25388fd   38.1562   
2  [-24.870956, -24.092632, -20.653963, -19.41104...  58b2aaa0   45.2859   
3  [-27.889421, -27.519794, -27.165262, -29.10350...  4cfc3a18   43.8306   
4  [-27.206915, -30.259186, -30.259186, -23.16495...  271f93f4   35.6256   

   is_iceberg  
0           0  
1           0  
2           1  
3           0  
4           0  


In [5]:
order_num = np.random.rand(train.shape[0])
train['order'] = order_num
train.sort_values('order', axis=0, inplace=True)
print(train.head())

                                                 band_1  \
1230  [-21.883642, -21.883698, -19.945553, -18.60667...   
183   [-18.310551, -14.242336, -14.242336, -15.37203...   
736   [-18.80862, -17.980827, -18.20948, -19.319889,...   
520   [-25.867931, -22.630978, -21.047417, -21.89143...   
651   [-20.482752, -20.961047999999998, -25.076342, ...   

                                                 band_2        id inc_angle  \
1230  [-28.273911, -27.209055, -25.681345, -25.96620...  0b44f893   39.5309   
183   [-25.670088, -27.097279, -27.097279, -27.09735...  434c4170   33.6352   
736   [-23.88929, -23.245657, -24.829346, -27.751968...  7fe65b97   37.2802   
520   [-27.736366, -29.674627, -29.67469, -28.460796...  c03c0809    37.237   
651   [-29.002129, -29.00218, -28.598166, -29.002287...  f613255d   39.6927   

      is_iceberg     order  
1230           0  0.000085  
183            1  0.000106  
736            1  0.000214  
520            1  0.001506  
651            0  0.00178

In [6]:
raw_y = train['is_iceberg'].tolist()
tensor_y = tf.one_hot(raw_y, depth=2)
with tf.Session() as sess:
    total_y = np.array(sess.run(tensor_y))
print(total_y.shape)

(1604, 2)


In [7]:
raw_x_1 = np.array(train['band_1'].tolist())
raw_x_2 = np.array(train['band_2'].tolist())
total_x = np.concatenate((raw_x_1, raw_x_2), axis = 1)
total_x = raw_x_1
print(total_x.shape)

(1604, 5625)


In [8]:
dev_x = total_x[0:300,:]
dev_y = total_y[0:300,:]
print(dev_x.shape)
train_x = total_x[300:,:]
train_y = total_y[300:,:]
print(train_x.shape)
print(train_y.shape)

(300, 5625)
(1304, 5625)
(1304, 2)


In [9]:
class MiniBatch(object):
    def __init__(self, x, y, batch_size):
        self.generator = self.slice_random(x, y, batch_size)
    
    def next_batch(self):
        try:
            return(next(self.generator))
        except StopIteration as e:
            return None, None
        
    @staticmethod    
    def slice_random(X, Y, mini_batch_size):
        tmp_X = X.copy()
        tmp_Y = Y.copy()
        cur_index = 0
        while(tmp_X.shape[0]>0):
            pick_size = tmp_X.shape[0] if tmp_X.shape[0]<mini_batch_size else mini_batch_size
            mini_batch_x = tmp_X[cur_index:cur_index + pick_size,:]
            mini_batch_y = tmp_Y[cur_index:cur_index + pick_size,:]
            cur_index += pick_size
            yield mini_batch_x, mini_batch_y

In [10]:
# test
test_mini_batch = MiniBatch(dev_x, dev_y, 50)


In [10]:
with tf.name_scope('reshape'):
    x_in = tf.placeholder(tf.float32)
    y_in = tf.placeholder(tf.float32)
    x = tf.image.resize_image_with_crop_or_pad(tf.reshape(x_in, [-1,75,75,1]), 256, 256) 
    y = tf.reshape(y_in, [-1, 2])

In [11]:
with tf.name_scope('resnet'):
    y_generator = resnet_model.imagenet_resnet_v2(18, 2, 'channels_last')
    y_ = y_generator(x, True)

In [12]:
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y))

In [13]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [14]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)

In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tf.add_to_collection('accuracy', accuracy)
    tf.add_to_collection('y_', y_)
    tf.add_to_collection('x_in', x_in)
    tf.add_to_collection('y_in', y_in)
    # writer = tf.summary.FileWriter('./logs',sess.graph)
    for i in range(1001):
        minibatch = MiniBatch(train_x, train_y, 50)
        x_batch,y_batch = minibatch.next_batch()
        if x_batch is None or x_batch.shape[0]<30:
            minibatch = MiniBatch(train_x, train_y, 50)
            x_batch,y_batch = minibatch.next_batch()
        # print('minibatch size {} and {}'.format(x_batch.shape, y_batch.shape))
        if i%100 == 0:
            train_accuracy = accuracy.eval(session=sess,
                                            feed_dict={
                                                x_in:train_x, y_in:train_y
                                            })
            print('step {}, training accuracy {}'.format(i, train_accuracy))
            print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))
            saver = tf.train.Saver()
            saved_path = saver.save(sess, './models/{}.ckpt'.format(i))
            print('model saved to {}'.format(saved_path))
        sess.run(optimizer, feed_dict={x_in:x_batch, y_in:y_batch})
    # writer.close()
    print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))
    

InvalidArgumentError: Incompatible shapes: [1304] vs. [5216]
	 [[Node: accuracy/Equal = Equal[T=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](accuracy/ArgMax, accuracy/ArgMax_1)]]

Caused by op 'accuracy/Equal', defined at:
  File "/home/barney_awesome_li/anaconda3/envs/tfcpu/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/barney_awesome_li/anaconda3/envs/tfcpu/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/barney_awesome_li/anaconda3/envs/tfcpu/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/barney_awesome_li/anaconda3/envs/tfcpu/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/barney_awesome_li/anaconda3/envs/tfcpu/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home

In [ ]:
# with tf.Session() as sess:
#     saver = tf.train.import_meta_graph('./models/1st.ckpt.meta')
#     saver.restore(sess, './models/1st.ckpt')
#     accuracy = tf.get_collection('accuracy')[0]
#     print('model restored')
#     print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in:mnist.test.images, y_in:mnist.test.labels})))

In [21]:
print(train_x.shape)
print(train_y.shape)


(1304, 5625)
(1304, 2)


In [26]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./logs',sess.graph)
    sess.run(cost, feed_dict={x_in:train_x[0:2], y_in:train_y[0:2]})
    #sess.run(accuracy, feed_dict={x_in:train_x[0:2], y_in:train_y[0:2]})
    writer.close()

InvalidArgumentError: logits and labels must be same size: logits_size=[8,2] labels_size=[2,2]
	 [[Node: cost/SoftmaxCrossEntropyWithLogits = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](cost/Reshape, cost/Reshape_1)]]

Caused by op 'cost/SoftmaxCrossEntropyWithLogits', defined at:
  File "/home/barney_awesome_li/anaconda3/envs/tfcpu/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/barney_awesome_li/anaconda3/envs/tfcpu/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/barney_awesome_li/anaconda3/envs/tfcpu/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/barney_awesome_li/anaconda3/envs/tfcpu/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/barney_awesome_li/anaconda3/envs/tfcpu/lib/python3.6/site-packages/ip

In [24]:
print(y_)

Tensor("resnet/final_dense:0", shape=(?, 2), dtype=float32)


In [25]:
print(y)

Tensor("reshape/Reshape_2:0", shape=(?, 2), dtype=float32)


In [29]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    yres = y.eval(session=sess, feed_dict={x_in:train_x[0:1], y_in:train_y[0:1]})
    y_res = y_.eval(session=sess, feed_dict={x_in:train_x[0:1], y_in:train_y[0:1]})
    print(yres)
    print(y_res)

[[ 0.  1.]]
[[-0.58980405  0.22809745]
 [-0.63758516  0.32957131]
 [-0.55162573  0.16295648]
 [-0.59601128  0.2735557 ]]
